In [1]:
# Required Libraries
import boto3
import botocore

import tarfile

import time

import os
from os import path
import shutil
import glob

In [45]:
#### Required functions

def delete(file='',folder=''):
    
    if folder:
        if path.exists(folder):
            shutil.rmtree(folder)
            print('------Deleted the folder: ',folder,' ------')
        else:
            return
    
    elif file:
        if path.exists(file):
            os.remove(file)
            print('------Deleted the file: ',file,' ------')
        else:
            return
    
    else:
        print('Invalid Path')

def download_file_from_s3(bucket_name='',file_name='',new_file_name=''):
    s3 = boto3.resource('s3')
    
    try:
        delete(file=new_file_name)
        s3.Bucket(bucket_name).download_file(file_name,new_file_name)
        print('---Downloaded Sucessfully---')
    
    except botocore.exceptions.ClientErrors as e:
        
        if e.response['Error']['Code'] == '404':
            print('The object does not exist')
        else:
            print('---Error in Downloading the S3 file---')

def extracting_file(file_name=''):
    
    if file_name:
        
        delete(folder=folder_name_local)
        
        my_tar = tarfile.open(file_name)
        my_tar.extractall()
        my_tar.close()
        delete(file=file_name_local)
        
        print('------ Extracted Successfully-----')
    else:
        print('Invalid address / file_name')

## Downloading and Extracting the CJP Data

In [46]:
start_time = time.time()

In [47]:
file_name_s3 = 'cjp-tables-af3a7aefbc195b871992.tar.gz'
file_name_local = 'cjp_data_file.tar.gz'
folder_name_local = 'cjp_tables'
s3_bucket_name = 'cjp-news-data'

In [48]:
%%time
downloading_time_start = time.time()
download_file_from_s3(bucket_name=s3_bucket_name,
                     file_name=file_name_s3,
                     new_file_name=file_name_local)
downloading_time_end = time.time()

---Downloaded Sucessfully---
Wall time: 1min 4s


In [49]:
%%time
download_and_extracting_time_start = time.time()
extracting_file(file_name=file_name_local)
download_and_extracting_time_end = time.time()

------Deleted the folder:  cjp_tables  ------
------Deleted the file:  cjp_data_file.tar.gz  ------
------ Extracted Successfully-----
Wall time: 1min 4s


## Extracting the required CJP NEWS Data From Downladed S3 file (2017-Present)

In [2]:
#%cd /Users/mchladek/Dev/article-tagging/lib/tagnews/

from req_file import load_all_data, download_data
import pandas as pd
import numpy as np
import datetime

SET GLOBAL VARIABLES

In [3]:
CATEGORY_RELEVANCE_MIN = 0.50

LOAD DATA

In [4]:
#%cd /Users/mchladek/Dev/chicago-justice-data/
data_loading_time_start = time.time()
newssource_df, articles_df, categories_df, trainedcategoryrelevance_df, trainedcoding_df, usercoding_df,  usercoding_categories_df, trainedlocation_df, trainedsentiment_df, trainedsentimententities_df = load_all_data()
data_loading_time_end = time.time()


news sources loaded. size: (39, 4)


C:\Users\sukes\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (1,3,10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


articles loaded. size: (883068, 11)
categories loaded. size: (40, 6)
trainedcategoryrelevance loaded. size: (1917198, 4)
trainedlocation loaded. size: (497718, 8)
trainedcoding loaded. size: (883068, 8)
usercoding loaded. size: (293737, 7)
usercoding_categories loaded. size: (158702, 3)
trainedsentiment loaded. size: (12660, 4)
trainedsentimententities loaded. size: (69610, 6)


### Looking into the Data - Practicum student code

In [5]:
newssource_df.head()

,source_id,source_name,short_name,legacy_feed_id
0,97,Unknown,unknown,NaN
1,98,ABC Local,abc-local,A
2,99,Beachwood Reporter,beachwood-reporter,C
3,100,CBS Local,cbs-local,L
4,101,Chicago Defender,chicago-defender,D


In [6]:
articles_df.head()

,id,feedname,url,orig_html,title,bodytext,relevant,created,last_modified,news_source_id,author
0,624597,NaN,https://abc7chicago.com/entertainment/disney-w...,"<div class=""body-text"">\n ORLANDO, Fla. -- Jus...",Disney World increases cost of annual passes a...,"ORLANDO, Fla. -- Just ahead of the opening of ...",t,2019-06-20 00:00:26.199741+00,2019-06-20 00:00:26.199761+00,98,NaN
1,624598,NaN,https://abc7chicago.com/miracle-on-metra-mans-...,"<div class=""body-text"">\n OSWEGO, Ill. (WLS) ...",Oswego man meets stranger who saved his life d...,"OSWEGO, Ill. (WLS) -- A stranger on a Metra tr...",t,2019-06-20 00:00:27.378267+00,2019-06-20 00:00:27.378287+00,98,NaN
2,624599,NaN,https://abc7chicago.com/sports/authorities-dav...,"<div class=""body-text"">\n Authorities in the D...",Authorities: David Ortiz not intended target o...,Authorities in the Dominican Republic have con...,t,2019-06-20 00:00:28.582828+00,2019-06-20 00:00:28.582847+00,98,NaN
3,624600,NaN,https://abc7chicago.com/drivers-under-influenc...,"<div class=""body-text"">\n CHICAGO (WLS) -- Acc...",Drivers under influence of marijuana pose chal...,"CHICAGO (WLS) -- According to AAA, drivers und...",t,2019-06-20 00:00:29.822443+00,2019-06-20 00:00:29.822463+00,98,Michelle Gallardo
4,624601,NaN,https://abc7chicago.com/food/fresh-food-market...,"<div class=""body-text"">\n CHICAGO (WLS) -- ""Th...","Fresh food market opens in Austin, offers loca...","CHICAGO (WLS) -- ""There's nothing like the wes...",t,2019-06-20 00:00:31.113093+00,2019-06-20 00:00:31.113114+00,98,NaN


In [7]:
articles_df.tail()

,id,feedname,url,orig_html,title,bodytext,relevant,created,last_modified,news_source_id,author
883063,624593,NaN,https://abc7chicago.com/society/-make-a-wish-s...,"<div class=""body-text"">\n CHICAGO (WLS) -- Wit...",Make-A-Wish surprises boy with sickle cell ane...,CHICAGO (WLS) -- With the help of Make-A-Wish ...,t,2019-06-20 00:00:21.195645+00,2019-06-20 00:00:21.195665+00,98,Ravi Baichwal
883064,624594,NaN,https://abc7chicago.com/society/chicago-junete...,"<div class=""body-text"">\n CHICAGO (WLS) -- Jun...",Chicago Juneteenth celebration honors African-...,CHICAGO (WLS) -- Juneteenth is the oldest know...,t,2019-06-20 00:00:22.454482+00,2019-06-20 00:00:22.454502+00,98,Stacey Baca
883065,624595,NaN,https://abc7chicago.com/entertainment/star-war...,"<div class=""body-text"">\n ANAHEIM, Calif. -- H...",Star Wars: Galaxy's Edge virtual queuing syste...,"ANAHEIM, Calif. -- Have you checked out Star W...",t,2019-06-20 00:00:23.726382+00,2019-06-20 00:00:23.726403+00,98,NaN
883066,624596,NaN,https://abc7chicago.com/2-dead-in-fiery-semi-c...,"<div class=""body-text"">\n RACINE COUNTY, Wis. ...","Racine County, Wisconsin, semi crash, explosio...","RACINE COUNTY, Wis. (WLS) -- At least two peop...",t,2019-06-20 00:00:24.962024+00,2019-06-20 00:00:24.962058+00,98,NaN
883067,624626,NaN,http://www.nbcchicago.com/news/local/authoriti...,"<div class=""articleText"" id=""at_0"" itemprop=""a...",Authorities Begin Piecing Together Events That...,Authorities are still working to piece togethe...,t,2019-06-20 00:02:23.219966+00,2019-06-20 00:02:23.219985+00,110,NaN


In [8]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883068 entries, 0 to 883067
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   id              883068 non-null  int64 
 1   feedname        292158 non-null  object
 2   url             883068 non-null  object
 3   orig_html       589271 non-null  object
 4   title           883042 non-null  object
 5   bodytext        877045 non-null  object
 6   relevant        883068 non-null  object
 7   created         883068 non-null  object
 8   last_modified   883068 non-null  object
 9   news_source_id  883068 non-null  int64 
 10  author          393602 non-null  object
dtypes: int64(2), object(9)
memory usage: 74.1+ MB


#### Coverting req columns to date type

In [9]:
# Converting last_modified and created columns to date-type
articles_df['last_modified'] = pd.to_datetime(articles_df['last_modified']).dt.tz_convert(None)
articles_df['created'] = pd.to_datetime(articles_df['created']).dt.tz_convert(None)

In [10]:
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 883068 entries, 0 to 883067
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   id              883068 non-null  int64         
 1   feedname        292158 non-null  object        
 2   url             883068 non-null  object        
 3   orig_html       589271 non-null  object        
 4   title           883042 non-null  object        
 5   bodytext        877045 non-null  object        
 6   relevant        883068 non-null  object        
 7   created         883068 non-null  datetime64[ns]
 8   last_modified   883068 non-null  datetime64[ns]
 9   news_source_id  883068 non-null  int64         
 10  author          393602 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 74.1+ MB


### Sorting the data-frame based on date - Descinding Order

In [11]:
articles_df.sort_values(by="created",ascending=False,inplace=True)

In [12]:
articles_df.head()

,id,feedname,url,orig_html,title,bodytext,relevant,created,last_modified,news_source_id,author
258808,883579,NaN,https://chicago.suntimes.com/2021/4/24/2240183...,"<div class=""c-entry-content piano-paywall-hide...",Fire allow critical own goal in loss to Atlant...,ATLANTA — Ezequiel Barco scored Atlanta’s firs...,t,2021-04-25 06:00:45.753547,2021-04-25 06:00:45.753565,112,Paul Newberry | AP
258807,883578,NaN,https://chicago.suntimes.com/2021/4/25/2239850...,"<div class=""c-entry-content piano-paywall-hide...","Horoscope for Sunday, April 25, 2021",## Moon Alert\n\nThere are no restrictions to...,t,2021-04-25 06:00:44.253835,2021-04-25 06:00:44.253853,112,Georgia Nicols
258806,883577,NaN,https://www.chicagonow.com/the-beeronaut/2021/...,"<div class=""post-content"">\n <figure>\n <img ...","Chicago's Week in Beer, April 26-29","![Chicago's Week in Beer, April 26-29][1] Atla...",t,2021-04-25 06:00:33.955987,2021-04-25 06:00:33.956004,104,Mark McDermott
258805,883576,NaN,https://chicago.cbslocal.com/2021/04/24/loyola...,"<div class=""featured-video""><div class=""anvato...","Lucas Williamson, 3 Other Loyola Ramblers’ Sen...",**CHICAGO (CBS) --** The Loyola Ramblers' chan...,t,2021-04-25 06:00:27.552906,2021-04-25 06:00:27.552920,100,CBS Chicago
258804,883575,NaN,https://abc7chicago.com/weather/chicago-weathe...,"<div class=""body-text"">\n CHICAGO (WLS) -- Sun...","Chicago Weather: Sunny, cooler by the lake Sunday","CHICAGO (WLS) -- Sunny, cooler by the lake Sun...",t,2021-04-25 06:00:25.198704,2021-04-25 06:00:25.198722,98,NaN


TRAINED CODING

Rename columns so column names are unique after merge

In [13]:
trainedcoding_df.head()

,id,date,model_info,relevance,article_id,sentiment,bin,sentiment_processed
0,1616804,2020-03-24 03:01:07.336104+00,tagnews 1.2.4,0.999997,30864,NaN,0.0,t
1,1616839,2020-03-25 03:00:32.490177+00,tagnews 1.2.4,0.999879,30899,NaN,0.0,t
2,1616842,2020-08-26 03:01:08.9806+00,tagnews 1.2.4,0.998977,30902,NaN,1.0,t
3,1616854,2020-08-26 03:01:09.418435+00,tagnews 1.2.4,0.996706,30914,NaN,1.0,t
4,1616758,2020-02-06 23:42:49.97428+00,tagnews 1.2.4,0.199123,30818,NaN,9.0,f


In [14]:
trainedcoding_df.columns = ['id', 'date', 'model_info', 'overall_relevance',
                            'article_id', 'sentiment', 'bin', 'sentiment_processed']

In [15]:
trainedcategoryrelevance_df.head()

,id,relevance,category_id,coding_id
0,4271682,0.997400,23,1703704
1,4271683,0.992532,26,1703704
2,4271684,0.969676,2,1703704
3,4271685,0.940466,17,1703704
4,4271686,0.317523,2,1703705


In [16]:
trainedcategoryrelevance_df.columns = ['id','cat_relevance','category_id','coding_id']

Replace category IDs with category titles

In [17]:
categories_df.head()

,id,title,abbreviation,created,active,kind
0,15,Juvenile,JUVE,2011-08-26 20:22:06.828537+00,t,other
1,16,Re-Entry,REEN,2011-08-26 20:22:20.557875+00,t,other
2,17,Violence,VIOL,2011-08-26 20:22:33.659219+00,t,other
3,19,Probation,PROB,2011-08-26 20:23:14.245505+00,t,other
4,20,Parole,PARL,2011-08-26 20:23:20.964542+00,t,other


In [18]:
for row in categories_df.itertuples():
    trainedcategoryrelevance_df.loc[trainedcategoryrelevance_df['category_id'] == row.id,
                                    'category_id'] = row.title

In [19]:
trainedcategoryrelevance_df.head()

,id,cat_relevance,category_id,coding_id
0,4271682,0.997400,Chicago Public Schools,1703704
1,4271683,0.992532,Homicides,1703704
2,4271684,0.969676,Chicago Police Department,1703704
3,4271685,0.940466,Violence,1703704
4,4271686,0.317523,Chicago Police Department,1703705


Get trained codings with relevant crime categories

In [20]:
# Can drop sentiment related columns from trained coding; can drop 'id' column from categories
rel_trained_df = pd.merge(left=trainedcoding_df.drop(columns=['sentiment','bin','sentiment_processed']),
                         right=trainedcategoryrelevance_df.drop(columns=['id']),
                         left_on='id',
                         right_on='coding_id')
rel_trained_df = rel_trained_df[(rel_trained_df['cat_relevance'] >= CATEGORY_RELEVANCE_MIN)]

display(rel_trained_df.sort_values(by='article_id').head(10))

,id,date,model_info,overall_relevance,article_id,cat_relevance,category_id,coding_id
1294405,1585986,2020-02-06 23:31:57.851733+00,tagnews 1.2.4,0.758854,1,0.758854,Gun Violence,1585986
1294409,1585988,2020-02-06 23:31:57.904407+00,tagnews 1.2.4,0.796053,3,0.796053,Chicago Police Department,1585988
1216552,1585991,2020-02-06 23:31:57.957061+00,tagnews 1.2.4,0.978682,6,0.978682,Unspecified Crime,1585991
1216556,1585994,2020-02-06 23:31:58.015216+00,tagnews 1.2.4,0.866723,9,0.716029,Homicides,1585994
1216555,1585994,2020-02-06 23:31:58.015216+00,tagnews 1.2.4,0.866723,9,0.866723,Unspecified Crime,1585994
1216562,1585997,2020-02-06 23:31:58.069588+00,tagnews 1.2.4,0.535168,12,0.535168,Chicago Police Department,1585997
1294657,1585999,2020-11-27 03:00:59.448363+00,tagnews 1.2.4,0.934560,14,0.934560,Chicago Police Department,1585999
1216535,1586000,2020-11-27 03:00:59.967033+00,tagnews 1.2.4,0.963884,15,0.963884,Cook County Jail,1586000
1216536,1586000,2020-11-27 03:00:59.967033+00,tagnews 1.2.4,0.963884,15,0.896525,Chicago Police Department,1586000
1216537,1586000,2020-11-27 03:00:59.967033+00,tagnews 1.2.4,0.963884,15,0.837398,Gun Violence,1586000


Reshape long format of categories into wide

In [21]:
rel_trained_df['category_id'] = rel_trained_df['category_id'].astype('str')
rel_trained_df = pd.pivot_table(rel_trained_df,
                                index=['overall_relevance','article_id','coding_id'],
                                values='category_id',
                                aggfunc=lambda x: ', '.join(x)).reset_index()
display(rel_trained_df.sort_values(by='article_id').head(10))

,overall_relevance,article_id,coding_id,category_id
80034,0.758854,1,1585986,Gun Violence
93111,0.796053,3,1585988,Chicago Police Department
213264,0.978682,6,1585991,Unspecified Crime
122159,0.866723,9,1585994,"Unspecified Crime, Homicides"
11052,0.535168,12,1585997,Chicago Police Department
162199,0.934560,14,1585999,Chicago Police Department
190510,0.963884,15,1586000,"Cook County Jail, Chicago Police Department, G..."
94152,0.798977,16,1586001,"Chicago Police Department, Burlgary"
217389,0.980782,17,1586002,Unspecified Crime
348788,0.999992,18,1586003,"Chicago Police Department, Beat Realignment / ..."


In [22]:
# Indicate what minimum relevance cutoff was used for categories
rel_trained_df['cat_relevance_min'] = CATEGORY_RELEVANCE_MIN

display(rel_trained_df.sort_values(by='article_id').head(10))

,overall_relevance,article_id,coding_id,category_id,cat_relevance_min
80034,0.758854,1,1585986,Gun Violence,0.5
93111,0.796053,3,1585988,Chicago Police Department,0.5
213264,0.978682,6,1585991,Unspecified Crime,0.5
122159,0.866723,9,1585994,"Unspecified Crime, Homicides",0.5
11052,0.535168,12,1585997,Chicago Police Department,0.5
162199,0.934560,14,1585999,Chicago Police Department,0.5
190510,0.963884,15,1586000,"Cook County Jail, Chicago Police Department, G...",0.5
94152,0.798977,16,1586001,"Chicago Police Department, Burlgary",0.5
217389,0.980782,17,1586002,Unspecified Crime,0.5
348788,0.999992,18,1586003,"Chicago Police Department, Beat Realignment / ...",0.5


Get trained location data

In [23]:
# We only want the locations indicated as is_best
best_trained_loc_df = trainedlocation_df[trainedlocation_df['is_best'] == 't']

rel_trained_loc_df = pd.merge(how='left',
                              left=rel_trained_df,
                              right=best_trained_loc_df.drop(columns=['id','neighborhood']),
                              left_on='coding_id',
                              right_on='coding_id')

display(rel_trained_loc_df.sort_values(by='article_id').head(10))

,overall_relevance,article_id,coding_id,category_id,cat_relevance_min,text,latitude,longitude,confidence,is_best
80297,0.758854,1,1585986,Gun Violence,0.5,NaN,NaN,NaN,NaN,NaN
93431,0.796053,3,1585988,Chicago Police Department,0.5,NaN,NaN,NaN,NaN,NaN
214468,0.978682,6,1585991,Unspecified Crime,0.5,NaN,NaN,NaN,NaN,NaN
122591,0.866723,9,1585994,"Unspecified Crime, Homicides",0.5,NaN,NaN,NaN,NaN,NaN
11074,0.535168,12,1585997,Chicago Police Department,0.5,NaN,NaN,NaN,NaN,NaN
162921,0.934560,14,1585999,Chicago Police Department,0.5,NaN,NaN,NaN,NaN,NaN
191369,0.963884,15,1586000,"Cook County Jail, Chicago Police Department, G...",0.5,NaN,NaN,NaN,NaN,NaN
94474,0.798977,16,1586001,"Chicago Police Department, Burlgary",0.5,Damen Avenue exit,41.595223,-87.665472,0.74,t
218612,0.980782,17,1586002,Unspecified Crime,0.5,NaN,NaN,NaN,NaN,NaN
351530,0.999992,18,1586003,"Chicago Police Department, Beat Realignment / ...",0.5,NaN,NaN,NaN,NaN,NaN


Clean up dataframe

In [24]:
# Clean up columns
rel_trained_loc_df = rel_trained_loc_df.drop(['is_best','coding_id'], axis=1)
rel_trained_loc_df.rename(columns = {'confidence':'loc_confidence',
                                     'category_id':'categories',
                                     'text':'loc_text'},
                          inplace = True)
display(rel_trained_loc_df.sort_values(by='article_id').head(10))

,overall_relevance,article_id,categories,cat_relevance_min,loc_text,latitude,longitude,loc_confidence
80297,0.758854,1,Gun Violence,0.5,NaN,NaN,NaN,NaN
93431,0.796053,3,Chicago Police Department,0.5,NaN,NaN,NaN,NaN
214468,0.978682,6,Unspecified Crime,0.5,NaN,NaN,NaN,NaN
122591,0.866723,9,"Unspecified Crime, Homicides",0.5,NaN,NaN,NaN,NaN
11074,0.535168,12,Chicago Police Department,0.5,NaN,NaN,NaN,NaN
162921,0.934560,14,Chicago Police Department,0.5,NaN,NaN,NaN,NaN
191369,0.963884,15,"Cook County Jail, Chicago Police Department, G...",0.5,NaN,NaN,NaN,NaN
94474,0.798977,16,"Chicago Police Department, Burlgary",0.5,Damen Avenue exit,41.595223,-87.665472,0.74
218612,0.980782,17,Unspecified Crime,0.5,NaN,NaN,NaN,NaN
351530,0.999992,18,"Chicago Police Department, Beat Realignment / ...",0.5,NaN,NaN,NaN,NaN


PREP ARTICLES

Merge news source names into articles DF

In [25]:
articles_merge_df = pd.merge(left=articles_df,
                      right=newssource_df,
                      left_on='news_source_id',
                      right_on='source_id')

In [26]:
# Can drop columns from newsource and article DFs that are no longer needed
articles_merge_df = articles_merge_df.drop(['feedname',
                                            'source_id',
                                            'legacy_feed_id',
                                            'news_source_id',
                                            'relevant',
                                            'orig_html',
                                            'bodytext'], axis=1)

In [27]:
# Reorder DF so news source name is first after id
articles_merge_df = articles_merge_df[['id','source_name','short_name','url','title',
                                       'created','last_modified','author']]

MERGE ARTICLES WITH TRAINED CODING CATEGORIES AND LOCATION

In [28]:
articles_trained_merge_df = pd.merge(left=articles_merge_df,
                                     right=rel_trained_loc_df,
                                     left_on='id',
                                     right_on='article_id')
print(articles_trained_merge_df.shape)

(367194, 16)


In [29]:
articles_trained_merge_df.tail()

,id,source_name,short_name,url,title,created,last_modified,author,overall_relevance,article_id,categories,cat_relevance_min,loc_text,latitude,longitude,loc_confidence
367189,12507,Chicago News Cooperative,chicago-news-cooperative,http://www.chicagonewscoop.org/warren-loud-sch...,Warren: Loud Schools Debate Misses Mark,2012-02-28 00:16:27.258493,2013-09-03 17:07:07.625589,NaN,0.999498,12507,Chicago Public Schools,0.5,NaN,NaN,NaN,NaN
367190,12506,Chicago News Cooperative,chicago-news-cooperative,http://www.chicagonewscoop.org/councils-sarge-...,Council’s “Sarge” on Plan to Quiet Meetings,2012-02-28 00:16:26.217300,2012-11-10 21:02:35.764959,NaN,0.784052,12506,Criminal Justice Policy,0.5,City Hall. She,41.884346,-87.631668,0.882
367191,12505,Chicago News Cooperative,chicago-news-cooperative,http://www.chicagonewscoop.org/homeless-immigr...,"Homeless Immigrants Alone, Adrift",2012-02-28 00:16:25.188685,2013-09-03 17:10:01.018327,NaN,0.986434,12505,Immigration,0.5,NaN,NaN,NaN,NaN
367192,12504,Chicago News Cooperative,chicago-news-cooperative,http://www.chicagonewscoop.org/warren-posner-p...,Warren: Posner Ponders Yet Again,2012-02-28 00:16:24.149505,2012-11-10 21:02:35.431678,NaN,0.951153,12504,"Chicago Police Department, Unspecified Crime",0.5,NaN,NaN,NaN,NaN
367193,12502,Chicago News Cooperative,chicago-news-cooperative,http://www.chicagonewscoop.org/leo-track-runni...,Leo Track Running Despite Hurdles,2012-02-28 00:16:22.102951,2012-11-10 21:02:35.455707,NaN,0.548729,12502,Chicago Public Schools,0.5,NaN,NaN,NaN,NaN


### Filtering the data frame : 2017 - Present

In [30]:
articles_df_2019_present = articles_trained_merge_df[articles_trained_merge_df['created']>datetime.datetime(2019,1,1)]

In [31]:
articles_df_2019_present.tail()

,id,source_name,short_name,url,title,created,last_modified,author,overall_relevance,article_id,categories,cat_relevance_min,loc_text,latitude,longitude,loc_confidence
351054,574530,fop-news,fop-news,https://fop7blog.org/news/2019/2/11/fop-attorn...,FOP Attorneys Will Demand Special Prosecutor I...,2019-02-12 14:00:06.835146,2019-02-12 14:00:06.835164,/Martin Preib,0.865878,574530,"Cook County State's Attorney's Office, Crimina...",0.5,NaN,NaN,NaN,NaN
351055,568652,fop-news,fop-news,https://fop7blog.org/news/2019/1/30/kimberly-f...,Kimberly Foxx: Prosecution or Revolution?,2019-01-30 12:00:05.981133,2019-01-30 12:00:05.981150,/Martin Preib,0.879766,568652,"Violence, Chicago Police Department",0.5,NaN,NaN,NaN,NaN
351056,566828,fop-news,fop-news,https://fop7blog.org/news/2019/1/25/federal-pr...,Federal Prosecutors Do What Kimberly Foxx Won'...,2019-01-25 22:00:06.103093,2019-01-25 22:00:06.103110,/Martin Preib,0.997161,566828,"Chicago Police Department, Cook County State's...",0.5,NaN,NaN,NaN,NaN
351057,564525,fop-news,fop-news,https://fop7blog.org/news/2019/1/20/tribgate-p...,Tribgate: Pattern of False Police Misconduct C...,2019-01-21 02:00:05.800759,2019-01-21 02:00:05.800776,/Martin Preib,0.999999,564525,"Police Misconduct, Chicago Police Department, ...",0.5,NaN,NaN,NaN,NaN
351058,564222,fop-news,fop-news,https://fop7blog.org/news/2019/1/19/judges-rul...,Judge’s Ruling Should Mean Stripped Officers A...,2019-01-19 22:00:20.912487,2019-01-19 22:00:20.912505,/Martin Preib,0.998902,564222,"Chicago Police Department, Police Misconduct, ...",0.5,McDonald matter. The,41.472653,-87.393335,0.74


### Refining the data-frame

In [32]:
articles_df_2019_present = articles_df_2019_present[['source_name','short_name',
                                                 'url','title','created','last_modified',
                                                 'categories','loc_text','latitude','longitude','loc_confidence']]

In [33]:
articles_df_2019_present.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135546 entries, 0 to 351058
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   source_name     135546 non-null  object        
 1   short_name      135546 non-null  object        
 2   url             135546 non-null  object        
 3   title           135544 non-null  object        
 4   created         135546 non-null  datetime64[ns]
 5   last_modified   135546 non-null  datetime64[ns]
 6   categories      135546 non-null  object        
 7   loc_text        43023 non-null   object        
 8   latitude        43023 non-null   float64       
 9   longitude       43023 non-null   float64       
 10  loc_confidence  43023 non-null   float64       
dtypes: datetime64[ns](2), float64(3), object(6)
memory usage: 12.4+ MB


In [34]:
articles_df_2019_present.head()

,source_name,short_name,url,title,created,last_modified,categories,loc_text,latitude,longitude,loc_confidence
0,Chicago Sun-Times,chicago-sun-times,https://chicago.suntimes.com/crime/2021/4/24/2...,Teen boy shot in Little Village,2021-04-25 02:00:44.803268,2021-04-25 02:00:44.803286,"Gun Violence, Chicago Police Department, Violence","3300 block of West 28th Street,",41.840826,-87.707982,0.8
1,Chicago Sun-Times,chicago-sun-times,https://chicago.suntimes.com/cubs/2021/4/24/22...,Cubs’ bullpen surrenders lead after Adbert Alz...,2021-04-25 02:00:43.288227,2021-04-25 02:00:43.288245,Burlgary,NaN,NaN,NaN,NaN
2,Chicago Sun-Times,chicago-sun-times,https://chicago.suntimes.com/crime/2021/4/24/2...,Armed robberies reported in Englewood,2021-04-25 00:00:50.304474,2021-04-25 00:00:50.304496,"Chicago Police Department, Robbery",5700 block of South Bishop Street;,41.790320,-87.661207,1.0
3,Chicago Sun-Times,chicago-sun-times,https://chicago.suntimes.com/crime/2021/4/24/2...,Man killed in Maywood shooting,2021-04-24 22:00:44.662127,2021-04-24 22:00:44.662145,"Homicides, Gun Violence","600 block of South 12th Avenue,",41.882132,-87.847640,0.8
4,Chicago Sun-Times,chicago-sun-times,https://chicago.suntimes.com/crime/2021/4/24/2...,Man charged in West Side murder of 7-year-old ...,2021-04-24 22:00:39.975204,2021-04-24 22:00:39.975221,"Gun Violence, Homicides, Juvenile, Chicago Pol...",NaN,NaN,NaN,NaN


In [35]:
articles_df_2019_present.tail()

,source_name,short_name,url,title,created,last_modified,categories,loc_text,latitude,longitude,loc_confidence
351054,fop-news,fop-news,https://fop7blog.org/news/2019/2/11/fop-attorn...,FOP Attorneys Will Demand Special Prosecutor I...,2019-02-12 14:00:06.835146,2019-02-12 14:00:06.835164,"Cook County State's Attorney's Office, Crimina...",NaN,NaN,NaN,NaN
351055,fop-news,fop-news,https://fop7blog.org/news/2019/1/30/kimberly-f...,Kimberly Foxx: Prosecution or Revolution?,2019-01-30 12:00:05.981133,2019-01-30 12:00:05.981150,"Violence, Chicago Police Department",NaN,NaN,NaN,NaN
351056,fop-news,fop-news,https://fop7blog.org/news/2019/1/25/federal-pr...,Federal Prosecutors Do What Kimberly Foxx Won'...,2019-01-25 22:00:06.103093,2019-01-25 22:00:06.103110,"Chicago Police Department, Cook County State's...",NaN,NaN,NaN,NaN
351057,fop-news,fop-news,https://fop7blog.org/news/2019/1/20/tribgate-p...,Tribgate: Pattern of False Police Misconduct C...,2019-01-21 02:00:05.800759,2019-01-21 02:00:05.800776,"Police Misconduct, Chicago Police Department, ...",NaN,NaN,NaN,NaN
351058,fop-news,fop-news,https://fop7blog.org/news/2019/1/19/judges-rul...,Judge’s Ruling Should Mean Stripped Officers A...,2019-01-19 22:00:20.912487,2019-01-19 22:00:20.912505,"Chicago Police Department, Police Misconduct, ...",McDonald matter. The,41.472653,-87.393335,0.74


EXPORT TO LOCAL FILE SYSTEM (.CSV)

In [36]:
articles_df_2019_present.to_csv("histroical_data.csv",index=False)

In [37]:
articles_df_2019_present.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 135546 entries, 0 to 351058
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   source_name     135546 non-null  object        
 1   short_name      135546 non-null  object        
 2   url             135546 non-null  object        
 3   title           135544 non-null  object        
 4   created         135546 non-null  datetime64[ns]
 5   last_modified   135546 non-null  datetime64[ns]
 6   categories      135546 non-null  object        
 7   loc_text        43023 non-null   object        
 8   latitude        43023 non-null   float64       
 9   longitude       43023 non-null   float64       
 10  loc_confidence  43023 non-null   float64       
dtypes: datetime64[ns](2), float64(3), object(6)
memory usage: 12.4+ MB


In [38]:
print("--- %s seconds ---" % (time.time() - start_time),' For the complete Process')
print("--- %s seconds ---" % (downloading_time_end - downloading_time_start),' For Downloading File from S3')
print("--- %s seconds ---" % (download_and_extracting_time_end - download_and_extracting_time_start),' For the Downlading and extraction File')
print("--- %s seconds ---" % (data_loading_time_end - data_loading_time_start),' For Loading the data into the notebook')

NameError: name 'start_time' is not defined

### Importing Final Data to Google Sheets

In [39]:

import pygsheets
import numpy as np

In [40]:
def write_gsheet(df,wkb_name):
    
    gc = pygsheets.authorize(service_account_file='GCP_credentials_for_gheets/chicagojusticeprojectscraping-7891c0e9a0db.json')
    sh = gc.open(wkb_name)
    sheet_num = str(sh.worksheets()[0])[17]
    rows_df,columns_df = df.shape
    
    if rows_df*columns_df >= 50000000:
        return False

    elif sheet_num == '1':
        wks = sh.add_worksheet("Sheet3",rows=2 ,cols=2,index=0)
        wks = sh.worksheet_by_title("Sheet1")
        sh.del_worksheet(wks)
        
        wks = sh.add_worksheet("Sheet2",rows=rows_df ,cols=columns_df,index=0)
        wks = sh.worksheet_by_title("Sheet2")
        wks.set_dataframe(df,(1,1))
        
        wks = sh.worksheet_by_title("Sheet3")
        sh.del_worksheet(wks)
        return True
    
    else:
        wks = sh.add_worksheet("Sheet3",rows=2 ,cols=2,index=0)
        wks = sh.worksheet_by_title("Sheet2")
        sh.del_worksheet(wks)
        
        wks = sh.add_worksheet("Sheet1",rows=rows_df ,cols=columns_df,index=0)
        wks = sh.worksheet_by_title("Sheet1")
        wks.set_dataframe(df,(1,1))
        
        wks = sh.worksheet_by_title("Sheet3")
        sh.del_worksheet(wks)
        return True
    

### Filtering Data From Jan 2021 - Present for Google Sheet


In [41]:
articles_df_present = articles_df_2019_present[articles_df_2019_present['created']>datetime.datetime(2021,1,1)]

In [42]:
articles_df_present = articles_df_present.drop(['short_name','last_modified','loc_confidence'],
                                              axis=1)

In [43]:
articles_df_present.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18441 entries, 0 to 349524
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   source_name  18441 non-null  object        
 1   url          18441 non-null  object        
 2   title        18441 non-null  object        
 3   created      18441 non-null  datetime64[ns]
 4   categories   18441 non-null  object        
 5   loc_text     6370 non-null   object        
 6   latitude     6370 non-null   float64       
 7   longitude    6370 non-null   float64       
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 1.3+ MB


In [56]:
articles_df_present.head()

,source_name,url,title,created,categories,loc_text,latitude,longitude
0,Chicago Sun-Times,https://chicago.suntimes.com/crime/2021/4/24/2...,Teen boy shot in Little Village,2021-04-25 02:00:44.803268,"Gun Violence, Chicago Police Department, Violence","3300 block of West 28th Street,",41.840826,-87.707982
1,Chicago Sun-Times,https://chicago.suntimes.com/cubs/2021/4/24/22...,Cubs’ bullpen surrenders lead after Adbert Alz...,2021-04-25 02:00:43.288227,Burlgary,NaN,0.000000,0.000000
2,Chicago Sun-Times,https://chicago.suntimes.com/crime/2021/4/24/2...,Armed robberies reported in Englewood,2021-04-25 00:00:50.304474,"Chicago Police Department, Robbery",5700 block of South Bishop Street;,41.790320,-87.661207
3,Chicago Sun-Times,https://chicago.suntimes.com/crime/2021/4/24/2...,Man killed in Maywood shooting,2021-04-24 22:00:44.662127,"Homicides, Gun Violence","600 block of South 12th Avenue,",41.882132,-87.847640
4,Chicago Sun-Times,https://chicago.suntimes.com/crime/2021/4/24/2...,Man charged in West Side murder of 7-year-old ...,2021-04-24 22:00:39.975204,"Gun Violence, Homicides, Juvenile, Chicago Pol...",NaN,0.000000,0.000000


In [54]:
articles_df_present[['latitude','longitude']] = articles_df_present[['latitude','longitude']].fillna(0) 

In [55]:
articles_df_present.isnull().sum()

source_name        0
url                0
title              0
created            0
categories         0
loc_text       12071
latitude           0
longitude          0
dtype: int64

In [58]:
%%time
write_gsheet(articles_df_present,'Scraped Data')

Wall time: 18.9 s


True